In [1]:
import datetime
import pandas as pd
from pymongo import MongoClient

In [3]:
client = MongoClient("mongodb://127.0.0.1:27017")
dbname = client['weather']
collection_name = dbname["weatherny"]
schema_name = dbname["schemadef"]

In [24]:
# data from https://www.noaa.gov/

df1 = pd.read_csv('demo_data/weather1.csv')
df1 = pd.concat([df1, pd.read_csv('demo_data/weather2.csv')])
df1 = pd.concat([df1, pd.read_csv('demo_data/weather3.csv')])
df1 = pd.concat([df1, pd.read_csv('demo_data/weather4.csv')])
df1 = pd.concat([df1, pd.read_csv('demo_data/weather5.csv')])
df1 = df1[df1['NAME']=='LAGUARDIA AIRPORT, NY US']
df1.drop(["TSUN", "WT01", "WT02", "WT03", "WT04", "WT05", "WT06", "WT08", "WT09", "WT11", "WT10", "WT13", "WT14",
         "WT15", "WT16", "WT17", "WT18", "WT19", "WT22"], axis=1, inplace=True)
print(len(df1))
df1.reset_index(inplace=True)
df1.fillna(0, inplace=True)
df1.to_csv('demo_data/weather_ny_2012-2022.csv', index=False)
df1.head()

3647


,index,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN
0,4945,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-10,5.4,0.0,0.0,0.0,0.0,24.9,28.3,21.1
1,4946,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-11,3.1,0.0,0.0,0.0,0.0,23.9,28.3,20.0
2,4947,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-12,3.7,0.0,0.0,0.0,0.0,26.5,31.7,22.2
3,4948,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-13,3.2,0.0,0.5,0.0,0.0,25.8,26.7,23.9
4,4949,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-14,3.7,0.0,0.0,0.0,0.0,24.7,28.3,22.2


In [28]:
df2 = pd.read_csv('demo_data/weather_ny_2012-2022.csv', index_col=0)
df2.head()

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN
index,,,,,,,,,,,
4945,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-10,5.4,0.0,0.0,0.0,0.0,24.9,28.3,21.1
4946,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-11,3.1,0.0,0.0,0.0,0.0,23.9,28.3,20.0
4947,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-12,3.7,0.0,0.0,0.0,0.0,26.5,31.7,22.2
4948,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-13,3.2,0.0,0.5,0.0,0.0,25.8,26.7,23.9
4949,USW00014732,"LAGUARDIA AIRPORT, NY US",2019-08-14,3.7,0.0,0.0,0.0,0.0,24.7,28.3,22.2


In [5]:
all_docs = []
for idx, row in df2.iterrows():
    d = {
         '_id': datetime.datetime.strptime(row['DATE'], "%Y-%m-%d"),
         #'date': datetime.datetime.strptime(row['DATE'], "%Y-%m-%d"),
         'awnd': row['AWND'],
         'pgtm': row['PGTM'],
         'prcp': row['PRCP'],
         'snow': row['SNOW'],
         'snwd': row['SNWD'],
         'tavg': row['TAVG'],
         'tmax': row['TMAX'],
         'tmin': row['TMIN']
        }
    all_docs.append(d)
collection_name.insert_many(all_docs)

In [6]:
inserts = pd.DataFrame(collection_name.find())
inserts.head()

,_id,awnd,pgtm,prcp,snow,snwd,tavg,tmax,tmin
0,2019-08-10,5.4,NaN,0.0,0.0,0.0,24.9,28.3,21.1
1,2019-08-11,3.1,NaN,0.0,0.0,0.0,23.9,28.3,20.0
2,2019-08-12,3.7,NaN,0.0,0.0,0.0,26.5,31.7,22.2
3,2019-08-13,3.2,NaN,0.5,0.0,0.0,25.8,26.7,23.9
4,2019-08-14,3.7,NaN,0.0,0.0,0.0,24.7,28.3,22.2


In [7]:
schema_name.insert_one({
    "table": 'weatherny',
    "fields": [
          { "name" : "_id",
            "type" : "Date",
            "hidden" : False },
          { "name" : "awnd",
            "type" : "Double",
            "hidden" : False },
          { "name" : "pgtm",
            "type" : "Double",
            "hidden" : False },
          { "name" : "prcp",
            "type" : "Double",
            "hidden" : False },
          { "name" : "snow",
            "type" : "Double",
            "hidden" : False },
          { "name" : "snwd",
            "type" : "Double",
            "hidden" : False },
          { "name" : "tavg",
            "type" : "Double",
            "hidden" : False },
          { "name" : "tmax",
            "type" : "Double",
            "hidden" : False },
          { "name" : "tmin",
            "type" : "Double",
            "hidden" : False },
        
        ]
    }
)